## Credit Card Dataset

### Below are three files that contain the customer’s transaction information and inventories in the credit card information.

a) CDW_SAPP_CUSTOMER.JSON: This file has the existing customer details.

b) CDW_SAPP_CREDITCARD.JSON: This file contains all credit card transaction information.

c) CDW_SAPP_BRANCH.JSON: Each branch’s information and details are recorded in this file.


In [17]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from dotenv import load_dotenv
from pyspark.sql.functions import*
from pyspark.sql.types import StringType, IntegerType, BooleanType, DoubleType
import pandas as pd
import os
import re


## Loading the environment variables

In [18]:
# Loading the .env file
# Loading as environment variable
load_dotenv()
user = os.environ.get("user")
pwd = os.environ.get("pwd")

## Creating the Spark Session

In [19]:
# Creating Spark Session
spark = SparkSession.builder.appName('Capstone').getOrCreate()

### Fetching the customer, branch and the creditcard data from the creditcard_capstone database

In [20]:
# Function to fetch all the creditcard, customer and branch data from the creditcard_capstone database
def get_creditcard_info(user, pwd):
    
# Getting the creditcard transaction data     
    df_transactions=spark.read.format("jdbc")    \
             .options(driver="com.mysql.cj.jdbc.Driver",\
                      user=user,\
                      password=pwd,\
                      url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                      dbtable = "creditcard_capstone.cdw_sapp_credit_card").load()
    
# Getting the customer data
    df_customers=spark.read.format("jdbc")    \
             .options(driver="com.mysql.cj.jdbc.Driver",\
                      user=user,\
                      password=pwd,\
                      url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                      dbtable = "creditcard_capstone.cdw_sapp_customer").load()
    
# Getting the branch data    
    df_branches=spark.read.format("jdbc")    \
             .options(driver="com.mysql.cj.jdbc.Driver",\
                      user=user,\
                      password=pwd,\
                      url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                      dbtable = "creditcard_capstone.cdw_sapp_branch").load()
    
# Returning the creditcard transaction, customer and branch data fetched from the creditcard_capstone database
    return df_transactions, df_customers, df_branches

## Extracting data from the files

In [21]:
# Function to extract the customer, branch and the creditcard data from the files
def extract():
    # Reading the customer json file into the spark dataframe
    df_customer = spark.read.json("json_files/cdw_sapp_custmer.json")

    # Reading the branch json file into the spark dataframe
    df_branch = spark.read.json('json_files/cdw_sapp_branch.json')

    # Reading the credit card json file into the pandas dataframe
    df_creditcard = pd.read_json("json_files/cdw_sapp_credit.json", lines=True)

    # Returning the customer, branch and the creditcard data after reading from thr input file
    return df_customer, df_branch, df_creditcard


## Transforming the data as per the mapping logic

In [22]:
# Function to transform the customer, branch and the creditcard data as per the mapping logic
def transform(df_customer, df_branch, df_creditcard):
# Customer data

    df_customer = df_customer.select(col("SSN").cast("int"), initcap(col("FIRST_NAME")).alias("FIRST_NAME"), lower(col("MIDDLE_NAME")).alias("MIDDLE_NAME"), \
                            initcap(col("LAST_NAME")).alias("LAST_NAME"), col("CREDIT_CARD_NO"), \
                            concat_ws(',', col("APT_NO"), col("STREET_NAME")).alias("FULL_STREET_ADDRESS"), \
                            col("CUST_CITY"), col("CUST_STATE"), col("CUST_COUNTRY"), col("CUST_ZIP").cast("int"), \
                            regexp_replace(col("CUST_PHONE"), r'^(\d{3})(\d{4})$', '(214)$1-$2').alias('CUST_PHONE'), \
                            col("CUST_EMAIL"), col("LAST_UPDATED").cast("timestamp"))

# Branch data

# Creating the temporary view for the branch data
    df_branch.createOrReplaceTempView("branchtable")

# Getting all the records from the branchtable created above and applying the mapping logic
    df_branch = spark.sql("SELECT CAST(BRANCH_CODE AS INT), BRANCH_NAME, BRANCH_STREET, BRANCH_CITY, \
                          BRANCH_STATE, CAST(IF(BRANCH_ZIP IS NULL, '99999', BRANCH_ZIP) AS INT) AS BRANCH_ZIP, \
                          CONCAT('(', SUBSTR(BRANCH_PHONE, 1, 3), ')', SUBSTR(BRANCH_PHONE, 4,3), '-', SUBSTR(BRANCH_PHONE, 7, 4)) AS BRANCH_PHONE, \
                          CAST(LAST_UPDATED AS TIMESTAMP) FROM BRANCHTABLE")


# Creditcard data

# Converting the data types and renaming the columns as per the mapping logic
    df_creditcard = df_creditcard.astype({"DAY":'str',"MONTH":'str',"YEAR":'str', "CREDIT_CARD_NO":'str'})
    df_creditcard.rename(columns={"CREDIT_CARD_NO" : "CUST_CC_NO"}, inplace=True)
    df_creditcard['DAY'] = df_creditcard['DAY'].str.zfill(2)
    df_creditcard['MONTH'] = df_creditcard['MONTH'].str.zfill(2)    

# Combining the day, month and year to form the TIMEID
    df_creditcard['TIMEID'] = df_creditcard['YEAR'] + df_creditcard['MONTH'] + df_creditcard['DAY']

# Dropping the Day, Month and Year columns from the creditcard data
    df_creditcard.drop(columns=['DAY', 'MONTH', 'YEAR'], axis = 1, inplace = True)

# Converting the credit card pandas dataframe into spark dataframe
    df_creditcard = spark.createDataFrame(df_creditcard)

# Converting the columns Branch Code, Cust SSN, Transaction ID to the interger data type
    df_creditcard= df_creditcard.withColumn("BRANCH_CODE", df_creditcard["BRANCH_CODE"].cast("int"))
    df_creditcard = df_creditcard.withColumn("CUST_SSN", df_creditcard["CUST_SSN"].cast("int"))
    df_creditcard = df_creditcard.withColumn("TRANSACTION_ID", df_creditcard["TRANSACTION_ID"].cast("int"))

    # Return the transformed customer, branch abd creditcard data
    return df_customer, df_branch, df_creditcard

## Loading the data into the database

In [23]:
# Function to load the customer, branch and the creditcard data to the database creditcard_capstone
def load(df_customer_data, df_branch_data, df_creditcard_data):
    # Writing the customer data to the customer database table
    df_customer_data.write.format("jdbc") \
                    .mode("overwrite") \
                    .option("truncate", "true") \
                    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
                    .option("createTableColumnTypes", "FIRST_NAME VARCHAR(30), MIDDLE_NAME VARCHAR(30), \
                    LAST_NAME VARCHAR(30), CREDIT_CARD_NO VARCHAR(20), FULL_STREET_ADDRESS VARCHAR(50), \
                    CUST_CITY VARCHAR(30), CUST_STATE VARCHAR(5), CUST_COUNTRY VARCHAR(30), \
                    CUST_PHONE VARCHAR(20), CUST_EMAIL VARCHAR(30)") \
                    .option("dbtable", "creditcard_capstone.CDW_SAPP_CUSTOMER") \
                    .option("user", user) \
                    .option("password", pwd) \
                    .save()

# Writing the branch data to the branch database table
    df_branch_data.write.format("jdbc") \
                    .mode("overwrite") \
                    .option("truncate", "true") \
                    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
                    .option("createTableColumnTypes", "BRANCH_NAME VARCHAR(30), BRANCH_STREET VARCHAR(50), \
                     BRANCH_CITY VARCHAR(30), BRANCH_STATE VARCHAR(5       ), BRANCH_PHONE VARCHAR(20)") \
                    .option("dbtable", "creditcard_capstone.CDW_SAPP_BRANCH") \
                    .option("user", user) \
                    .option("password", pwd) \
                    .save()


# Writing the creditcard data to the creditcard database table
    df_creditcard_data.write.format("jdbc") \
                    .mode("overwrite") \
                    .option("truncate", "true") \
                    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
                    .option("createTableColumnTypes", "CUST_CC_NO VARCHAR(20), TIMEID VARCHAR(10), \
                    TRANSACTION_TYPE VARCHAR(30)") \
                    .option("dbtable", "creditcard_capstone.CDW_SAPP_CREDIT_CARD") \
                    .option("user", user) \
                    .option("password", pwd) \
                    .save()


## Displaying Schema

In [24]:
# Function to print the schema in a tree structure for the dataframe data provided
def print_schema(df):
    df.printSchema()
    return ""
   

## Displaying data types

In [25]:
# Function to print the data types of the columns in the dataframe for the given data
def print_data_types(df):
    print(df.dtypes)
    return ""
      

## Displaying the customer, branch and creditcard data

In [26]:
# Function to display the records with all the information 
def display_data(df_type):
    df_type.show(5)
    return ""


## Extract, Transform and Load (ETL)

In [27]:
def extract_transform_load():    

# Extract, transform and load data to the creditcard_capstone database

    # Extract
    print("Extraction started")

    # Extract the customer, branch and creditcard transaction data
    df_customer, df_branch, df_creditcard = extract()

    print("\nExtraction completed")

    # Transform
    print("\nTransform started")

    # Transform the customer, branch and the creditcard transaction data
    df_customer_data, df_branch_data, df_creditcard_data = transform(df_customer, df_branch, df_creditcard)

    print("\nTransform completed")

    # Displaying the schemas after transformation
    print("\nDisplaying the schema after transformation")
    print("\nCustomer schema after transformation:")
    print_schema(df_customer_data)
    print("\nBranch schema after transformation:")
    print(print_schema(df_branch_data))
    print("\nCreditcard schema after transformation:")
    print(print_data_types(df_creditcard_data))

    # Displaying the customer, branch and creditcard transaction data after transformation
    # and before loading it to the creditcard_capstone database
    print("\nDisplaying the data before loading to the database")
    
    # Displaying customer data
    print("\nCustomer data:")
    display_data(df_customer_data)

    # Displaying the branch data
    print("\nBranch data:")
    display_data(df_branch_data)

    # Displaying the creditcard data
    print("\nCreditcard data:")
    display_data(df_creditcard_data)

    # Loading
    print("\nLoading started")

    # Load the customer, branch and the creditcard transaction data
    load(df_customer_data, df_branch_data, df_creditcard_data)
    
    print("\nLoading completed")    

## Business Requirements - ETL

## 1. Functional Requirements - Load Credit Card Database (SQL)

### Req-1.1 Data Extraction and Transformation with Python and PySpark

Functional Requirement 1.1

a) For “Credit Card System,” create a Python and PySpark SQL
program to read/extract the following JSON files according 
to the specifications found in the mapping document.
1. CDW_SAPP_BRANCH.JSON
2. CDW_SAPP_CREDITCARD.JSON
3. CDW_SAPP_CUSTOMER.JSON

Note: Data Engineers will be required to transform the data based on the
requirements found in the Mapping Document.

Hint: [You can use PYSQL “select statement query” or simple Pyspark
RDD].

### Req-1.2 Data loading into Database

### Function Requirement 1.2

Once PySpark reads data from JSON files, and then utilizes Python,
PySpark, and Python modules to load data into RDBMS(SQL), perform
the following:

a) Create a Database in SQL(MariaDB), named
“creditcard_capstone.”

b) Create a Python and Pyspark Program to load/write the “Credit
Card System Data” into RDBMS(creditcard_capstone).

Tables should be created by the following names in RDBMS:

CDW_SAPP_BRANCH

CDW_SAPP_CREDIT_CARD

CDW_SAPP_CUSTOMER

In [28]:
# Checking if the customer, branch and creditcard table exists in the creditcard_capstone database
# Extract, transform and load the customer, branch and creditcard data only if table does not exists in the database
df_table=spark.read.format("jdbc")    \
             .options(driver="com.mysql.cj.jdbc.Driver",\
                      user=user,\
                      password=pwd,\
                      url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                      dbtable = "information_schema.tables").load().filter("table_schema = 'creditcard_capstone'")
# 
if df_table.isEmpty():
    extract_transform_load()

## 2. Functional Requirements - Application Front-End

Once data is loaded into the database, we need a front-end (console) to see/display data. For
that, create a console-based Python program to satisfy System Requirements 2 (2.1 and 2.2).

### 2.1 Transaction Details Module

### Req-2.1 Transaction Details Module

### Functional Requirements 2.1 


#### 1) Used to display the transactions made by customers living in a given zip code for a given month and year. 
#### Order by day in descending order.

In [32]:
# Fetching all the creditcard transaction, customer and branch data from the creditcard_capstone database
df_transactions, df_customers, df_branches = get_creditcard_info(user, pwd)

# Creating a temporary view for the creditcard transaction data
df_transactions.createOrReplaceTempView("creditview")

# Creating a temporary view for the customer data
df_customers.createOrReplaceTempView("customerview")

# Creating a temporary view for the branch data
df_branches.createOrReplaceTempView("branchview")

### Function to display the transactions made by customers living in a given zip code for a given month and year. 
### Order by day in descending order.

In [33]:
# Transaction module

# Function to display the transactions made by customers living in a
# given zip code for a given month and year. Order by day in
# descending order.

def transaction_customer_for_zip_month_year():
    while True:

     # Getting the zipcode as input
        zipcode = input("\nEnter the ZIP code in 5 digits: ")
        zipcode = zipcode.strip()

# Validating the zipcode

        # Checking if the zipcode has only numbers and is of length 5
        if zipcode.isdigit() and len(zipcode) == 5:
           zipcode = int(zipcode)
            # If the zipcode has only numbers and is of length 5 then it is a valid zipcode
           print('Valid zipcode')
           break
        else:
    # If the zipcode does not have only numbers then it is a invalid zipcode
           print('\nInvalid entry. Try again.')

    while True:

    # Getting the month as input
        month = input("\nEnter the month in 2 digits: ")
        month = month.strip().lstrip('0')

# Validating the month

        # Checking whether the month contains only numbers
        if month.isdigit():
    # Checking whether the month is in the range of 1 to 12
            if int(month) in range(0,13):
                # If the month is only numbers and in the range of 1 and 12 then it is a valid month
                print('Valid month')
                break
            else:
                # If the month is not in the range of 1 and 12 then it is an invalid month
                print("\nInvalid month. Try Again.") 
        else:
            # If the month does not have only numbers then it is an invalid entry
            print("\nInvalid entry. Try Again.")         

    while True:
        # Getting the year as input
        year = input("\nEnter the year in 4 digits: ")
        year = year.strip()

# Validating the year

# Checking whether the year contains only numbers and is of length 4
        if year.isdigit() and len(year) == 4:
            # If year is a number and has 4 digits then it is a valid year
            print('Valid year')
            break   
        else:
            # If the year does not have only numbers then it is an invalid year
            print("\nInvalid entry. Try Again.")


    # Fetching the records from the creditview and customerview 
    # to display the transactions made by customers living in a
    # given zip code for a given month and year. Order by day in
    # descending order.

    df_result = spark.sql("SELECT * from CREDITVIEW \
    JOIN CUSTOMERVIEW \
    ON CUSTOMERVIEW.credit_card_no = CREDITVIEW.cust_cc_no \
    WHERE CUSTOMERVIEW.cust_zip = {} \
    AND  MONTH(to_date(TIMEID, 'yyyyMMdd')) = '{}' \
    AND  YEAR(to_date(CREDITVIEW.TIMEID, 'yyyyMMdd')) = '{}' \
    ORDER BY DAY(to_date(TIMEID, 'yyyyMMdd')) DESC".format(zipcode, month, year))


# Displaying the customer transaction details
    if df_result.isEmpty():
        print('\nNo data matching criteria.')

    df_result.show()


In [35]:
# Function to display the transactions made by customers living in a
# given zip code for a given month and year. Order by day in
# descending order

transaction_customer_for_zip_month_year()

Valid zipcode
Valid month
Valid year

No data matching criteria.
+--------------+----------+--------+-----------+----------------+-----------------+------+---+----------+-----------+---------+--------------+-------------------+---------+----------+------------+--------+----------+----------+------------+
|TRANSACTION_ID|CUST_CC_NO|CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TIMEID|SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|CREDIT_CARD_NO|FULL_STREET_ADDRESS|CUST_CITY|CUST_STATE|CUST_COUNTRY|CUST_ZIP|CUST_PHONE|CUST_EMAIL|LAST_UPDATED|
+--------------+----------+--------+-----------+----------------+-----------------+------+---+----------+-----------+---------+--------------+-------------------+---------+----------+------------+--------+----------+----------+------------+
+--------------+----------+--------+-----------+----------------+-----------------+------+---+----------+-----------+---------+--------------+-------------------+---------+----------+------------+--------+-------

#### 2) Used to display the number and total values of transactions for a given type.


### Function to display the number and total values of transactions for a given transaction type.

In [36]:
# Function to display the number and total values of transactions for a given transaction type
def number_and_total_values_of_transactions():

    # Getting the unique transaction types from the creditview
    tran_type = spark.sql("SELECT DISTINCT TRANSACTION_TYPE FROM CREDITVIEW")
    # Converting it to Pandas dataframe to have it as a list to validate 
    df_tran = tran_type.toPandas()
    # Getting the unique values of the transaction type as a list
    tran_type = pd.unique(df_tran['TRANSACTION_TYPE'])

    while True:

        # Getting the transaction type as input
        transaction_type = input("\nEnter Transaction Type: ")
        transaction_type = transaction_type.strip()

        # Validating whether the entered transaction type exists in the database
        if transaction_type.title() in tran_type:
        # If exists then it is a valid transaction type
            print("\nValid transaction type\n")
            break
        else:
        # If the transaction type does not exists in the database then it is not a valid one 
            print("\nNo transaction type found. Try Again")

    # Displaying the number and total values of transactions for a given transaction type.
    df_result = spark.sql("SELECT TRANSACTION_TYPE, COUNT(TRANSACTION_ID) AS NUMBER_OF_TRANSACTIONS, \
    SUM(TRANSACTION_VALUE) AS TRANSACTION_VALUE FROM CREDITVIEW \
    WHERE TRANSACTION_TYPE = '{}' \
    GROUP BY TRANSACTION_TYPE".format(transaction_type))


    # Displaying the transaction details for a given transaction type
    if df_result.isEmpty():
        print('\nNo data matching criteria.')

    df_result.show()

In [37]:
# Function to display the number and total values of transactions for a given transaction type
number_and_total_values_of_transactions()

No transaction type found. Try Again
No transaction type found. Try Again

Valid transaction type

+----------------+----------------------+-----------------+
|TRANSACTION_TYPE|NUMBER_OF_TRANSACTIONS|TRANSACTION_VALUE|
+----------------+----------------------+-----------------+
|             Gas|                  6605|336059.2600000005|
+----------------+----------------------+-----------------+



#### 3) Used to display the number and total values of transactions for branches in a given state.

### Function to display the number and total values of transactions for branches in a given state.

In [42]:
# Function to display the number and total values of transactions for branches in a given state.
def number_and_total_transaction_values_for_branches():

    # Getting the unique states from the branchview.  
    br_state = spark.sql("SELECT DISTINCT BRANCH_STATE FROM BRANCHVIEW")
    # Converting it to Pandas dataframe to have it as a list to validate.
    df_state = br_state.toPandas()
    # Getting the unique values of the state as a list.
    branch_states = pd.unique(df_state['BRANCH_STATE'])

    while True:
        # Getting the state as input.
        state = input('\nEnter state in 2 letters: ')
        # Removing the leading and trailing spaces.
        state = state.strip().upper()

        # Checking whether the state has only letters and the length is 2.
        if state.isalpha() and len(state) == 2:
            # Converting the state to upper case.
            if state in branch_states:
            # If the State exists in the database then it is a valid state.
                print('\nBranch(es) found for given state')
                break
            else:
                # If State does not exist then no branches exist.
                print('\nNo branches found. Try again.')
        else:
            # If the State entered does not have only letters or not of length 2 then it
            # is not a valid State.
            print('\nInvalid entry. Try again.')


    # Display the number and total values of transactions for
    # branches in a given state.
    df_result = spark.sql("SELECT COUNT(TRANSACTION_ID) AS NUMBER_OF_TRANSACTIONS, \
    SUM(TRANSACTION_VALUE) AS TRANSACTION_VALUE FROM CREDITVIEW \
    JOIN BRANCHVIEW ON CREDITVIEW.BRANCH_CODE = BRANCHVIEW.BRANCH_CODE \
    WHERE BRANCHVIEW.BRANCH_STATE ='{}' \
    GROUP BY BRANCHVIEW.BRANCH_CODE".format(state))


    print("\nThe number and total values of transactions for branches in the State {}:".format(state))

    # Displaying the transactions for the branches
    if df_result.isEmpty():
        print('\nNo data matching criteria.')
        
    df_result.show()

In [43]:
# Function to display the number and total values of transactions for branches in a given state.
number_and_total_transaction_values_for_branches()


No branches found. Try again.

Branch(es) found for given state

The number and total values of transactions for branches in the State NY:
+----------------------+------------------+
|NUMBER_OF_TRANSACTIONS| TRANSACTION_VALUE|
+----------------------+------------------+
|                   416| 21698.66999999998|
|                   434|22766.419999999987|
|                   431|          21749.91|
|                   398|18968.330000000005|
|                   393|20524.620000000024|
|                   431|          21809.74|
|                   400|20257.589999999993|
|                   383| 20710.98999999999|
|                   412|21723.450000000004|
|                   445|23507.660000000033|
+----------------------+------------------+

